In [30]:
import os
import time
import numpy as np

import csv


"""For downloading the data set, only needed once"""
#! pip install -q kaggle
#from google.colab import files
"""For the kaggle.json file if using an online processor"""
#files.upload()
#! mkdir ~/.kaggle
#! cp kaggle.json ~/.kaggle/
#! chmod 600 ~/.kaggle/kaggle.json
#!kaggle datasets download yamaerenay/spotify-dataset-19212020-160k-tracks -p /content/sample_data/ --unzip



'For the kaggle.json file if using an online processor'

In [37]:
"""Edit DATADIR to whereever you are storing tracks.csv"""
DATADIR = '/Users/harrison/Harrison/CS4641/Project/tracks.csv'
#CATEGORIES = ["Rock", "Pop", "Jazz", "Rap", "Hip Hop", "EDM", "Funk"]
results = []

counter = 0
min_tempo = float('inf')
max_tempo = float('-inf')

min_loud = float('inf')
max_loud = float('-inf')

with open(DATADIR) as csvfile:
    reader = csv.reader(csvfile)#, quoting=csv.QUOTE_NONNUMERIC) # change contents to floats
    for row in reader: # each row is a list
        row = np.delete(row, 0,0) # id
        row = np.delete(row, 0,0) # name 
        row = np.delete(row, 0,0) # popularity
        row = np.delete(row, 0,0) # duration_ms
        row = np.delete(row, 0,0) # explicit
        row = np.delete(row, 0,0) # artists
        row = np.delete(row, 0,0) # id_artists
        row = np.delete(row, 0,0) # release_date
        row = np.delete(row, 2,0) # key
        row = np.delete(row, 10,0) # time signature
        if not row[0] == 'danceability': #ignore the first row of labels
            row = row.astype(np.float)
            results.append(row)
            
            current_tempo = row[9]
            if current_tempo > max_tempo:
                max_tempo = current_tempo
            if current_tempo < min_tempo:
                min_tempo = current_tempo
                
            current_loud = row[2]
            if current_loud > max_loud:
                max_loud = current_loud
            if current_loud < min_loud:
                min_loud = current_loud


for i in range(len(results)):
    results[i][9] = (results[i][9] - min_tempo) / (max_tempo - min_tempo)
    results[i][2] = (results[i][2] - min_loud) / (max_loud - min_loud)

In [38]:
print(results[0])
#results = np.delete(results,0,0)
print(results[1])

[0.645      0.445      0.71374816 1.         0.451      0.674
 0.744      0.151      0.127      0.42556447]
[0.695      0.263      0.57917279 1.         0.957      0.797
 0.         0.148      0.655      0.41402949]


In [39]:
class KMeans(object):
    
    def pairwise_dist(self, x, y):
        z = np.linalg.norm(x - y)
        return z
    
    def _init_centers(self, points, K, **kwargs): # [3 pts]
        np.random.seed(1)

        i = 0
        size = len(points)
        centers = []
        randoms = []
        while i < K:
            random = np.random.randint(0, size)
            if not randoms.__contains__(random):
                randoms.append(random)
                centers.append(points[random])
                i = i + 1

        centers = np.array(centers)
        return centers

    def _update_assignment(self, centers, points): # [6 pts]
        np.random.seed(1)

        cluster_idx = []
        for i in range(len(points)):
            nearest_center = 0
            held_distance = float('inf')
            for j in range(len(centers)):
                num_dist = self.pairwise_dist(points[i], centers[j])
                if num_dist < held_distance:
                    nearest_center = j
                    held_distance = num_dist
            cluster_idx.append(nearest_center)

        return cluster_idx
    
    def _update_centers(self, old_centers, cluster_idx, points): # [8 pts]
        np.random.seed(1)

        centers = []
        for i in range(len(old_centers)):
            counter = 0
            storage = points[0] - points[0]
            for j in range(len(points)):
                if cluster_idx[j] == i:
                    storage = storage + points[j]
                    counter = counter + 1
            storage = storage / counter
            centers.append(storage)

        centers = np.array(centers)
        return centers
    
    def _get_loss(self, centers, cluster_idx, points): # [8 pts]
        np.random.seed(1)
        
        total = 0
        for i in range(len(points)):
            cluster = cluster_idx[i]

            difference = points[i] - centers[cluster]
            square = np.dot(difference.T, difference)
            total = total + square
        return total
    
    def __call__(self, points, K, max_iters=100, abs_tol=1e-16, rel_tol=1e-16, verbose=False, **kwargs): # no need to implement
        """
        Args:
            points: NxD numpy array, where N is # points and D is the dimensionality
            K: number of clusters
            max_iters: maximum number of iterations (Hint: You could change it when debugging)
            abs_tol: convergence criteria w.r.t absolute change of loss
            rel_tol: convergence criteria w.r.t relative change of loss
            verbose: boolean to set whether method should print loss (Hint: helpful for debugging)
            kwargs: any additional arguments you want
        Return:
            cluster assignments: Nx1 int numpy array
            cluster centers: K x D numpy array, the centers
            loss: final loss value of the objective function of KMeans
        """
        centers = self._init_centers(points, K, **kwargs)
        for it in range(max_iters):
            cluster_idx = self._update_assignment(centers, points)
            centers = self._update_centers(centers, cluster_idx, points)
            loss = self._get_loss(centers, cluster_idx, points)
            K = centers.shape[0]
            if it:
                diff = np.abs(prev_loss - loss)
                if diff < abs_tol and diff / prev_loss < rel_tol:
                    break
            prev_loss = loss
            if verbose:
                print('iter %d, loss: %.4f' % (it, loss))
        return cluster_idx, centers, loss

    def find_optimal_num_clusters(self, data, max_K=25,iteration = 100): # [5 pts]
        np.random.seed(1)
        """Plots loss values for different number of clusters in K-Means

        Args:
            data: input data of shape(samples, 2)
            max_K: number of clusters
        Return:
            losses: a list, which includes the loss values for different number of clusters in K-Means
            Plot loss values against number of clusters
        """
        losses = []
        #for i in range(max_K):
            #if i > 0:
        cluster_id, centers, loss = self.__call__(data, max_K)#i)
        print("Final Centers: ")
        print(centers)
        losses.append(loss)
        print("Cluster IDs: ")
        print(cluster_id)
        print("Losses: ")
        print(losses)
        return losses

In [40]:
cluster_idx, centers, loss = KMeans().__call__(results, 25)

cluster_idx is an N x 1 array. N corresponding to the index of the song in the results (N x D) array. 

In [41]:
cluster_idx.insert(0, "Cluster")

In [42]:
with open('to_write.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for i in cluster_idx:
        row = [i]
        writer.writerow(row)